# Multiple Light Source

Simulating multiple light sources is straightforward by exploiting the linearity
of the radiance field: We simply run a separate simulation for each light source
and aggregate the results, for instance using addition.

If all light sources use the same model (e.g. `SphericalLightSource`) one can
simply reuse the same pipeline and issue multiple tasks. If however the light
sources differ in their nature, say a point source and an extend track, you
will have to create a separate pipeline for each type. Fortunately, pipeline
stages may be shared between pipelines (as long as they do not run in parallel)
and multiple pipelines can be managed by the same scheduler requiring only
minimal changes compared to the previous examples as we will show in this
notebook.

To keep things simple we will combine a `SphericalLightSource` and a
`PencilLightSource` with an ordinary `VolumeForwardTracer`.

In [ ]:
# Uncomment this, if you run this on Google Colab
# !sudo apt-get install -y libnvidia-gl-550 vulkan-tools
# !pip install git+https://github.com/tkerscher/theia

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import hephaistos as hp
import hephaistos.pipeline as pl

import theia
import theia.units as u

# print device we run this on
print(hp.getCurrentDevice())

The following steps should feel similar from the `01_volume_tracing` example.
Only difference being, that here we create two pipelines, one for each light
source.

In [ ]:
# Load medium to GPU
# How to create media should be already known so we will use a premade one
medium = theia.testing.WaterTestModel().createMedium()
store = theia.material.MaterialStore([], media=[medium])

# shared stages
rng = theia.random.PhiloxRNG(key=0xABBA)
response = theia.response.HistogramHitResponse(
    theia.response.UniformValueResponse(),
    nBins=100,
    binSize=5.0 * u.ns,
)
target = theia.target.SphereTarget(
    position=(-1.0, 0.0, 0.0) * u.m,
    radius=50.0 * u.cm,
)
wavelength = theia.light.ConstWavelengthSource(500.0 * u.nm)
sharedStages = [target, wavelength, response, rng]

# the two light sources, we config them later using tasks
sphereLight = theia.light.SphericalLightSource()
pencilLight = theia.light.PencilLightSource()

# share settings between tracer via dict
batchSize = 512 * 1024
tracerParams = {
    "nScattering": 10,
    "medium": store.media["water"],
}
# create two separate tracers for each light source
# Tracers cannot be shared as they contain compiled code
sphereTracer = theia.trace.VolumeForwardTracer(
    batchSize,
    sphereLight,
    *sharedStages,
    **tracerParams,
)
pencilTracer = theia.trace.VolumeForwardTracer(
    batchSize,
    pencilLight,
    *sharedStages,
    **tracerParams,
)
# set up auto advance
rng.autoAdvance = max(sphereTracer.nRNGSamples, pencilTracer.nRNGSamples)

# Finally, create the two pipelines
spherePipeline = pl.Pipeline(sphereTracer.collectStages())
pencilPipeline = pl.Pipeline(pencilTracer.collectStages())

With the pipelines ready we can now create our scheduler. To use multiple
pipelines with a scheduler you have to pass as a list of tuples, each assigning
a name to the corresponding pipeline. This name is later used to specify on
which pipeline to run a task.

Note that because we shared `response` between both tracers, our process function
does not need to know which pipeline produced the results.

In [ ]:
hists = []
def process(config: int, batch: int, args: None) -> None:
    hists.append(response.result(config).copy())

pipelineMap = {
    "sphere": spherePipeline,
    "pencil": pencilPipeline,
}
scheduler = pl.PipelineScheduler(pipelineMap, processFn=process)

After everything is set up, we can finally issue some work to the pipelines.
Unlike in previous examples, tasks now have to specify on which pipeline to run.

In [ ]:
tasks = [
    # tuple with pipeline name + params to apply
    ("sphere", {
        "lightSource__budget": 1e5,
        "lightSource__timeRange": (0.0, 10.0) * u.ns,
        "lightSource__position": (1.0, 0.0, 0.0) * u.m,
    }),
    # issue another 4 batches with the same params
    ("sphere", {}),
    ("sphere", {}),
    ("sphere", {}),
    ("sphere", {}),
    # next is the pencil light source
    ("pencil", {
        "lightSource__position": (0.0, 1.0, 0.0) * u.m,
        "lightSource__direction": (-0.8, -0.36, -0.24),
        "lightSource__budget": 5e6,
        "lightSource__timeRange": (8.0, 8.5) * u.ns,
    }),
    ("pencil", {}),
    ("pencil", {}),
    ("pencil", {}),
    ("pencil", {}),
]

# submit work
scheduler.schedule(tasks)
scheduler.wait()

Last thing to do is to make some nice plots

In [ ]:
t = np.arange(response.nBins) * response.binSize + response.t0
# aggregate the five batches per light source
sphereSignal = np.mean(hists[:5], 0)
pencilSignal = np.mean(hists[5:], 0)

plt.figure()
plt.step(t, sphereSignal, label="sphere")
plt.step(t, pencilSignal, label="pencil")
plt.yscale("log")
plt.legend()
plt.show()